In [188]:
%load_ext autoreload
%autoreload 2
from corsikaio import CorsikaParticleFile
import numpy as np
from tqdm import tqdm
from read_corsika import corsika_data_files, read_file 
from utilities import (pdg_mass_maps, angle_and_p2, 
                       mass_pdg_level, kinetic_energy, final_values)

from read_data import values_from_corsika_file, Corsika2Dict
from save2hdf5 import save_dict_to_hdf5

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [189]:
data_files = corsika_data_files("34_simple_run")


In [192]:
pdgs_to_get = [-12, 12, -13, 13, -14, 14]
theta = 30

In [194]:
result = []
for file in tqdm(data_files, total = len(data_files)):
    corsika_dict = Corsika2Dict(file, theta)
    result.append(corsika_dict.distr_dict(pdgs_to_get))

100%|██████████| 600/600 [06:51<00:00,  1.46it/s]


In [134]:
save_dict_to_hdf5(res, "15_test.hdf5")

In [187]:
arg = [1.3, -1.4, 0.8, -0.05, -2, -100, 100, 1.001]
np.clip(arg, a_min=-1.0, a_max=1.0)

array([ 1.  , -1.  ,  0.8 , -0.05, -1.  , -1.  ,  1.  ,  1.  ])

In [95]:
vals = values_from_corsika_file(data_files[0])


In [96]:
vals[0]

array([14011., 14012., 13041., ..., 68534.,  6532.,  6533.], dtype=float32)

In [99]:
vals = values_from_corsika_file(data_files[0])


In [100]:
pdgs, obs_levels, angles, kin_energy = final_values(*vals,
                                                    zenith_angle = 30)

In [115]:
obs_range = np.unique(obs_levels)
print(obs_range)
pdg_range = np.unique(pdgs)
print(pdg_range)

[0 1 2 3]
[-2212 -2112  -321  -211   -14   -13   -12    12    13    14   130   211
   321  2112  2212]


[1, 4, 5]


In [109]:
corsika_results_dict = {}

for pdg in [-13, 13]:
    corsika_results_dict[pdg] = {}
    for obslev_id in range(4):
        corsika_results_dict[pdg][obslev_id] = {'theta [rad]': None, 
                                                'energy [GeV]': None}
    
    
        filtered_ids = np.where((pdgs == pdg) & (obs_levels == obslev_id))[0]
        corsika_results_dict[pdg][obslev_id]['theta [rad]'] = angles[filtered_ids]
        corsika_results_dict[pdg][obslev_id]['energy [GeV]'] = kin_energy[filtered_ids]
        # corsika_results_dict[pdg][obslev_id]['pid'] = np.array(ids)

In [108]:
angles[np.where((pdgs == 13) & (obs_levels == 1))[0]]
kin_energy[np.where((pdgs == 13) & (obs_levels == 1))[0]]


array([20.62654   ,  0.79413015,  0.6001453 , ...,  0.9913832 ,
        1.776424  ,  1.5336462 ], dtype=float32)

In [ ]:
len(np.where((pdgs == 13) & (obs_levels == 1))[0])

In [74]:
results = []
tot_size = 0
with CorsikaParticleFile(data_files[0]) as f:
        for e in f:
            results.append(e.particles)
            tot_size += results[-1].size
print(tot_size)  

  
particle_desc = np.empty(tot_size, dtype=np.float32)
particle_px = np.empty(tot_size, dtype=np.float32)
particle_py = np.empty(tot_size, dtype=np.float32)
particle_pz = np.empty(tot_size, dtype=np.float32)
particle_x = np.empty(tot_size, dtype=np.float32)
particle_y = np.empty(tot_size, dtype=np.float32)
particle_t = np.empty(tot_size, dtype=np.float32)

325437


In [76]:

prev_ind = 0
for result in results:
    next_ind = prev_ind + len(result)
    particle_desc[prev_ind:next_ind] = result["particle_description"]
    particle_px[prev_ind:next_ind] = result["px"]
    particle_py[prev_ind:next_ind] = result["py"]
    particle_pz[prev_ind:next_ind] = result["pz"]
    particle_x[prev_ind:next_ind] = result["x"]
    particle_y[prev_ind:next_ind] = result["y"]
    particle_t[prev_ind:next_ind] = result["t"]
    prev_ind = next_ind 
        

In [73]:
pdgs, obs_levels, angles, kin_energy = final_values(particle_desc, 
                                                    particle_px, 
                                                    particle_py, 
                                                    particle_pz,
                                                    30)

In [65]:
masses, pdgs, obs_levels = mass_pdg_level(particle_desc)
angles, p2 = angle_and_p2(particle_px, particle_py, particle_pz, 30)
kin_energy = kinetic_energy(masses, p2)

In [53]:
corsika_pids = np.array(particle_desc // 1000, dtype=np.int32)
obs_level_id = np.array(particle_desc % 10, dtype=np.int32)

In [44]:
obs_level_id[0:10]

array([1, 2, 1, 2, 1, 2, 3, 4, 1, 1], dtype=int32)

In [54]:
masses = mass_map[corsika_pids]

In [55]:
angles, p2 = angle_and_p2(particle_px, particle_py, particle_pz, 30)

In [57]:
p2/(np.sqrt(p2+masses**2) + masses)

array([0.4781239, 0.3138018, 0.6620777, ..., 0.9372801, 1.5336462,
       0.6235557], dtype=float32)

In [99]:
np.where(corsika_pids == 69)[0]

array([4, 5, 6, 7, 9])

In [95]:
corsika_id_dict = {13: 6, -13: 5, 
                   211: 8, -211: 9, 
                   2212: 14, -2212: 15,
                   2112: 13, -2112: 25,
                   12: 66, -12: 67,
                   14: 68, -14: 69,
                   321: 11, -321: 12, 130 : 10}

my_dict = corsika_id_dict

reverse_dict = {}
for key, value in my_dict.items():
    if value in reverse_dict:
        reverse_dict[value].append(key)
    else:
        reverse_dict[value] = [key]

sorted(reverse_dict.keys())

[5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 25, 66, 67, 68, 69]

In [92]:
sorted([int(i) for i in set(corsika_pids)])

[5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 25, 66, 67, 68, 69]

In [96]:

# min_value = min(corsika_id_dict.values())
max_value = max(reverse_dict.keys())

pdg_from_corsika_ids =  np.zeros(max_value + 1)

for i, value in enumerate(pdg_from_corsika_ids):
    if i in reverse_dict:
        pdg_from_corsika_ids[i] = reverse_dict[i][0]
        
print(pdg_from_corsika_ids)        

[    0.     0.     0.     0.     0.   -13.    13.     0.   211.  -211.
   130.   321.  -321.  2112.  2212. -2212.     0.     0.     0.     0.
     0.     0.     0.     0.     0. -2112.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.    12.   -12.    14.   -14.]


In [102]:
from particle import Particle

mass_array = np.zeros_like(pdg_from_corsika_ids)

for i, pdg in enumerate(pdg_from_corsika_ids):
    try:
        mass_array[i] = Particle.from_pdgid(pdg).mass/1e3
    except Exception as ex:
        mass_array[i] = 0.

In [103]:
mass_array

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.10565838, 0.10565838, 0.        , 0.13957039, 0.13957039,
       0.497611  , 0.493677  , 0.493677  , 0.93956542, 0.93827209,
       0.93827209, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.93956542, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [80]:
print(min_value)
print(max_value)

5
69


In [67]:
# stack the three vectors into a 2D array
p = np.stack((particle_px, particle_py, particle_pz), axis=1)

# compute p2 as the sum of squares of px, py, and pz
p2 = np.sum(p**2, axis = 1)

# compute the norm of each row of the array
p_norm = np.sqrt(p2)

# normalize the rows of the array to get the direction vectors
ndir = p / p_norm[:, np.newaxis]

In [71]:
theta = 0
ndir0 = np.array([np.sin(np.radians(theta)), 0, np.cos(np.radians(theta))])

In [72]:
# compute the angle between ndir and ndir0 in radians
angles = np.arccos(np.sum(ndir * ndir0, axis=1))

In [62]:
np.sum(p**2, axis = 1)

array([1.1258231, 0.6873345, 1.6824777, ..., 0.8784941, 2.676156 ,
       0.5205895], dtype=float32)

In [53]:
# norms[0:10, np.newaxis]
theta = 0
unit_vec = np.array([np.sin(np.radians(theta)), 0, np.cos(np.radians(theta))])


In [55]:
# ndir[0:10] * unit_vec

dot_products = np.sum(ndir * unit_vec, axis=1)

# compute the angle between ndir and unit_vec in radians
angles = np.arccos(dot_products)

In [57]:
len(angles)

325437

In [7]:
results[0]

array([(14011., -0.29297963, -2.03848034e-02,  1.0195934 , -3.6714497e+05, -8.48890430e+03,  19298.924),
       (14012., -0.22892138, -1.59278065e-02,  0.79666543, -7.9949488e+05, -1.84854453e+04,  43775.188),
       (13041.,  0.5342469 , -1.52775159e-04,  1.181972  , -2.1839584e+04,  1.46143201e+03,  16763.32 ),
       (13042.,  0.5342469 , -1.52775159e-04,  1.181972  , -8.4516586e+04,  1.39680481e+03,  39363.293),
       (69031.,  0.12680763,  2.05083206e-01,  0.23262906, -1.8821764e+04,  8.37309609e+04,  16989.459),
       (69032.,  0.12680763,  2.05083206e-01,  0.23262906, -3.4943609e+04,  5.24525438e+05,  41010.36 ),
       (69033.,  0.12680763,  2.05083206e-01,  0.23262906, -6.7187297e+04,  1.40611438e+06,  89052.164),
       (69034.,  0.12680763,  2.05083206e-01,  0.23262906, -8.3309148e+04,  1.84690888e+06, 113073.06 ),
       ( 6031.,  0.38629082,  1.19824648e-01,  0.7555698 , -2.2962684e+04,  1.74629492e+04,  16066.232),
       (69541.,  0.14722891,  1.25581091e-02,  0.389363

In [33]:
filtered_px = 5.554
filtered_py = -0.1
filtered_pz = 0.18

filtered_pT = np.sqrt(filtered_px**2 + filtered_py**2)          
filtered_ptot = np.sqrt(filtered_px**2 + filtered_py**2 + filtered_pz**2)


filtered_theta = np.arctan2(filtered_pT, filtered_pz)
filtered_phi = np.arctan2(filtered_py, filtered_px)

np.arctan2(filtered_pT, filtered_pz)
filtered_phi = np.arctan2(filtered_py, filtered_px)
final_dir = np.array([np.cos(filtered_phi) * np.sin(filtered_theta),
                                        np.sin(filtered_phi) * np.sin(filtered_theta),
                                        np.cos(filtered_theta)]).T

In [34]:
print(final_dir)

ppp = np.array([filtered_px, filtered_py, filtered_pz])
print(ppp/np.linalg.norm(ppp))

[ 0.99931344 -0.01799268  0.03238682]
[ 0.99931344 -0.01799268  0.03238682]
